# Phishing Detection Using BERT

## Authors
- Eli Fajardo (NVIDIA)
- Gorkem Batmaz (NVIDIA)
- Bartley Richardson, PhD (NVIDIA)

## Table of Contents 
* Introduction
* List of datasets used
* Reading in the datasets
* Initialize CLX Phishing Detection and BERT model
* Training - CLAIR FRAUDULENT EMAILS dataset
* Evaluation of CLAIR Test Set
* Training with the the SPAM_ASSASSIN dataset
* Evaluation of the SPAM_ASSASSIN Test Set
* Training with all three datasets CLAIR+SPAM_ASSASSIN+ENRON
* Evaluation of the Test Set of CLAIR+SPAM_ASSASSIN+ENRON Datasets
* References

## Introduction
Phishing is a method used by fraudsters/hackers to obtain sensitive information from email users by pretending to be from legitimate institutions/people.
Various machine learning methods are in use to detect and filter phishing/spam emails. 
In this notebook, we show how to train a *BERT language model and analyse the performance on various datasets. We have fine-tuned a pre-trained BERT model with a classification layer using HuggingFace library. 
*BERT stands for Bidirectional Encoder Representations from Transformers. The paper can be found [here.](https://arxiv.org/pdf/1810.04805.pdf)
This notebook will be updated with a much faster GPU tokenizer

## Datasets used
* [CLAIR-Fraudulent E-mail Corpus](https://www.kaggle.com/rtatman/fraudulent-email-corpus)
* [SPAM_ASSASSIN Dataset](https://spamassassin.apache.org/old/publiccorpus/)
* [Enron Emails](https://www.cs.cmu.edu/~enron/)

### Required Libraries

In [ ]:
import cudf
from cuml.preprocessing.model_selection import train_test_split
from clx.analytics.phishing_detector import PhishingDetector
import s3fs
from os import path

## Reading the files

In [ ]:
CLAIR_TSV = "Phishing_Dataset_Clair_Collection.tsv"
SPAM_TSV = "spam_assassin_spam_200_20021010.tsv"
EASY_HAM_TSV = "spam_assassin_easyham_200_20021010.tsv"
HARD_HAM_TSV = "spam_assassin_hardham_200_20021010.tsv"
ENRON_TSV = "enron_10000.tsv"

S3_BASE_PATH = "rapidsai-data/cyber/clx"

In [ ]:
# Clair dataset
if not path.exists(CLAIR_TSV):
    fs = s3fs.S3FileSystem(anon=True)
    fs.get(S3_BASE_PATH + "/" + CLAIR_TSV, CLAIR_TSV)
    
dfclair = cudf.read_csv(CLAIR_TSV, delimiter='\t', header=None, names=['label', 'email'])

In [ ]:
# Phishing emails of the SPAM ASSASSIN dataset
if not path.exists(SPAM_TSV):
    fs = s3fs.S3FileSystem(anon=True)
    fs.get(S3_BASE_PATH + "/" + SPAM_TSV, SPAM_TSV)
 
dfspam = cudf.read_csv(SPAM_TSV, delimiter='\t', header=None, names=['label', 'email'])

In [ ]:
# Benign emails of the SPAM ASSASSIN dataset
if not path.exists(EASY_HAM_TSV):
    fs = s3fs.S3FileSystem(anon=True)
    fs.get(S3_BASE_PATH + "/" + EASY_HAM_TSV, EASY_HAM_TSV)
    
dfeasyham = cudf.read_csv(EASY_HAM_TSV, delimiter='\t', header=None, names=['label', 'email'])

In [ ]:
# Benign emails of the SPAM ASSASSIN dataset that are easy to be confused with phishing emails
if not path.exists(HARD_HAM_TSV):
    fs = s3fs.S3FileSystem(anon=True)
    fs.get(S3_BASE_PATH + "/" + HARD_HAM_TSV, HARD_HAM_TSV)

dfhardham = cudf.read_csv(HARD_HAM_TSV, delimiter='\t', header=None, names=['label', 'email'])

In [ ]:
# Benign Enron emails
if not path.exists(ENRON_TSV):
    fs = s3fs.S3FileSystem(anon=True)
    fs.get(S3_BASE_PATH + "/" + ENRON_TSV, ENRON_TSV)

dfenron = cudf.read_csv(ENRON_TSV, delimiter='\t', header=None, names=['label', 'email'])

In [ ]:
# The files contain the first 200 words of each email. The model uses only the first 128 words.

## Initialize/Load BERT model

In [ ]:
phish_detect = PhishingDetector()
phish_detect.init_model()

# init_model can also load pre-trained model by passing it model directory path

## Training - CLAIR FRAUDULENT EMAILS DATASET

Split the dataset into training and test sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dfclair, 'label', train_size=0.8)

In [ ]:
phish_detect.train_model(X_train, y_train, epochs=1)

## Evaluation of CLAIR Test Set

## Training with SPAM_ASSASSIN dataset

Merging the spam assasin dataset

In [ ]:
df_assassin = cudf.concat([dfhardham,dfeasyham,dfspam], ignore_index=True)

Split the dataset into train and test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_assassin, 'label', train_size=0.8)

In [ ]:
phish_detect.train_model(X_train, y_train, epochs=1)

## Evaluation of the SPAM_ASSASSIN Test Set

In [ ]:
phish_detect.evaluate_model(X_test, y_test)

## Training with CLAIR+SPAM_ASSASSIN datasets

Merge the two datasets and split as train and test sets

In [ ]:
df_total = cudf.concat([dfhardham,dfeasyham,dfspam,dfclair],ignore_index=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_total, 'label', train_size=0.8)

In [ ]:
phish_detect.train_model(X_train, y_train, epochs=1)

## Evaluation of the Test Set of CLAIR+SPAM_ASSASSIN Datasets

In [ ]:
phish_detect.evaluate_model(X_test, y_test)

## Training with all three datasets (CLAIR+SPAM_ASSASSIN+ENRON)

Merge all the datasets, split into training and test set and then tokenize the emails

In [ ]:
df_total = cudf.concat([dfhardham,dfeasyham,dfspam,dfclair,dfenron],ignore_index=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_total, 'label', train_size=0.8)

In [ ]:
phish_detect.train_model(X_train, y_train, epochs=1)

## Evaluation of the Test Set of CLAIR+SPAM_ASSASSIN+ENRON Datasets

In [ ]:
phish_detect.evaluate_model(X_test, y_test)

# References
* https://github.com/huggingface/transformers/tree/master/examples#
* https://www.depends-on-the-definition.com/named-entity-recognition-with-bert/
* https://github.com/ThilinaRajapakse/pytorch-transformers-classification
* https://mccormickml.com/2019/07/22/BERT-fine-tuning/